In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from matplotlib.patches import Rectangle
from pyiem.plot import MapPlot, nwssnow
from pyiem.nws.vtec import NWS_COLORS
from pyiem.util import get_dbconn, utc, get_sqlalchemy_conn
from pyiem.reference import EPSG
from pyproj import Transformer
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
STORM_NUMBER = 10
TITLE = "21-22 February 2022"
SUBTITLE = "8 AM 23 February 2022"
SETPOINT_LOCS = {}
sts = utc(2022, 2, 22, 0)
ets = utc(2022, 2, 23, 20, 0)
# Get available data
ctx = {'coop': 'yes', 't': 'state', 'sz': 60, 'z': 'yes', 'f': 'linear', 'v': 'snow', 'wfo': 'DMX'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, 'IA')
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2021-2022 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df2)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [4]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        geo = gpd.points_from_xy(x=[SETPOINT_LOCS[sp][0]], y=[SETPOINT_LOCS[sp][1]], crs=df.crs)
        df.loc[10000 + sp] = {"geo": geo[0], "val": val, "used_for_analysis": True,
                             "plotme": True}
    df["geo"] = gpd.GeoSeries(df["geo"], crs=df.crs)

def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz/2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz/2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1

def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets)
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0, ), ("Snow Squall Warning", ), loc=1).set_zorder(1000)

In [11]:
def main():
    setpoints = [
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = []
    if cull:
        df.loc[df['nwsli'].isin(cull), USEME] = False
    hardcode = [
        ('IA-FL-3', 2),
        ('IA-JH-20', 0),
    ]
    for nwsli, val in hardcode:
        df.loc[df['nwsli'] == nwsli, 'val'] = val

    ctx['csector'] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0, upper=6)
    # draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename='220223.png')
    mp.close()

In [12]:
main()

     state  wfo   val       lon      lat                               geo  \
0       WI  DLH  33.5  -90.9300  46.6600     POINT (690419.172 223580.004)   
2       WI  DLH  29.7  -91.2400  46.7700     POINT (665590.564 233134.526)   
3       WI  DLH  28.0  -90.8900  46.5200     POINT (695232.104 208436.105)   
7       WY  CYS  24.0 -107.2600  41.4000   POINT (-604868.802 -373528.415)   
8       AK  AFC  24.0 -150.0000  62.7900  POINT (-2328710.864 2799066.759)   
...    ...  ...   ...       ...      ...                               ...   
3454    MO  LSX   0.0  -91.6436  39.5247    POINT (716047.601 -572490.354)   
3455    AR  SHV   0.0  -93.2336  33.2505   POINT (631719.398 -1279148.420)   
3456    MO  LSX   0.0  -90.5778  38.6339    POINT (817516.657 -661200.623)   
3458    MO  LSX   0.0  -90.4167  38.2500    POINT (836022.188 -702039.950)   
3460    MO  LSX   0.0  -91.9000  39.1756    POINT (697751.676 -613320.244)   

      used_for_analysis  nwsli  plotme source  xcell  ycell  
0

<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.